In [2]:
import graphviz as gv

def readAdjl(fn, haslabels=False, weighted=False, sep="|"):
  with open(fn) as f:
    labels = None
    if haslabels:
      labels = f.readline().strip().split()
    L = []
    for line in f:
      if weighted:
        L.append([tuple(map(int, p.split(sep))) for p in line.strip().split()])
        # line => "1|3 2|5 4|4" ==> [(1, 3), (2, 5), (4, 4)]
      else:
        L.append(list(map(int, line.strip().split()))) # "1 3 5" => [1, 3, 5]
        # L.append([int(x) for x in line.strip().split()])
  return L, labels

def adjlShow(L, labels=None, directed=False, weighted=False, path=[],
             layout="sfdp", node_labels=None):
    g = gv.Digraph("G") if directed else gv.Graph("G")
    g.graph_attr["layout"] = layout
    g.edge_attr["color"] = "gray"
    g.node_attr["color"] = "orangered"
    g.node_attr["width"] = "0.1"
    g.node_attr["height"] = "0.1"
    g.node_attr["fontsize"] = "8"
    g.node_attr["fontcolor"] = "mediumslateblue"
    g.node_attr["fontname"] = "monospace"
    g.edge_attr["fontsize"] = "8"
    g.edge_attr["fontname"] = "monospace"
    n = len(L)
    for u in range(n):
        if labels:
            label = labels[u]
        else:
            label = str(u)
        if node_labels and u in node_labels:
            label = node_labels[u]
        g.node(str(u), label)

    added = set()
    for v, u in enumerate(path):
        if u is not None:
            if weighted:
                for vi, w in L[u]:
                    if vi == v:
                        break
                g.edge(str(u), str(v), str(w), dir="forward", penwidth="2", color="orange")
            else:
                g.edge(str(u), str(v), dir="forward", penwidth="2", color="orange")
            added.add(f"{u},{v}")
            added.add(f"{v},{u}")

    if weighted:
        for u in range(n):
            for v, w in L[u]:
                if not directed and f"{u},{v}" not in added:
                    added.add(f"{u},{v}")
                    added.add(f"{v},{u}")
                    g.edge(str(u), str(v), str(w))
                elif directed:
                    g.edge(str(u), str(v), str(w))
    else:
        for u in range(n):
            for v in L[u]:
                if not directed and f"{u},{v}" not in added:
                    added.add(f"{u},{v}")
                    added.add(f"{v},{u}")
                    g.edge(str(u), str(v))
                elif directed:
                    g.edge(str(u), str(v))

    return g

In [3]:
%%file 2.in
1|28
2|16 6|14
3|12
4|22 6|18
5|25
0|10
4|24

Writing 2.in


In [4]:
# Primero, carga tu grafo y define las etiquetas (si es necesario)
G, _ = readAdjl("2.in", weighted=True)

# Luego, puedes definir etiquetas personalizadas para los nodos si lo deseas
etiquetas = {
    0: "A",
    1: "B",
    2: "C",
    3: "D",
    4: "E",
    5: "F",
    6: "G"
}

for i, edges in enumerate(G):
    if etiquetas and i in etiquetas:
      label = etiquetas[i]
    else:
      label = str(i)
    print(f"Nodo {label}: {edges}")

# Finalmente, llama a la función adjlShow pasando las etiquetas y etiquetas personalizadas
adjlShow(G, _, weighted=True, node_labels=etiquetas)



Error: remove_overlap: Graphviz not built with triangulation library


Nodo A: [(1, 28)]
Nodo B: [(2, 16), (6, 14)]
Nodo C: [(3, 12)]
Nodo D: [(4, 22), (6, 18)]
Nodo E: [(5, 25)]
Nodo F: [(0, 10)]
Nodo G: [(4, 24)]


CalledProcessError: Command '[WindowsPath('dot'), '-Kdot', '-Tsvg']' returned non-zero exit status 1. [stderr: 'Error: remove_overlap: Graphviz not built with triangulation library\n']

In [5]:
def obtenerMSTPorKrukalFuerzaBruta(graph):
    numNodos = len(graph)
    bordes = []

    # saco todas las arista, su conexión y peso
    for u in range(numNodos):
        for v, peso in graph[u]:
            bordes.append((u, v, peso))

    # ordeno por peso
    bordes.sort(key=lambda edge: edge[2])

    # Acá gurdaré las aristas del mst
    mst = []
    padre = list(range(numNodos))

    def find(nodoBuscado):
        if padre[nodoBuscado] == nodoBuscado:
            return nodoBuscado
        return find(padre[nodoBuscado])

    def union(u, v):
        padreU = find(u)
        padreV = find(v)

        if padreU != padreV:
            padre[padreU] = padreV
            return True

        return False

    for u, v, peso in bordes:
        if u < numNodos and v < numNodos:
            if union(u, v) == True:
                mst.append((u, v, peso))

    return mst

In [8]:
mst = obtenerMSTPorKrukalFuerzaBruta(G)

print("Minimum Spanning Tree (MST):", mst)

costoMinimo = 0

for borde in mst:
    u, v, peso = borde
    print(f"Nodo {chr(u + 65)} con {chr(v + 65)} tienen peso: {peso}")
    costoMinimo += peso

print(f"Costo minimo: {costoMinimo}")

Minimum Spanning Tree (MST): [(5, 0, 10), (2, 3, 12), (1, 6, 14), (1, 2, 16), (3, 4, 22), (4, 5, 25)]
Nodo F con A tienen peso: 10
Nodo C con D tienen peso: 12
Nodo B con G tienen peso: 14
Nodo B con C tienen peso: 16
Nodo D con E tienen peso: 22
Nodo E con F tienen peso: 25
Costo minimo: 99
